## Race stratified ROC curves with 95\% CI using fast deLong method

In [ ]:
import matplotlib as mpl
import matplotlib.colors
import os 
import pandas as pd
import numpy as np
from os.path import join as j_
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report, confusion_matrix
import matplotlib.font_manager as fm


cdict = {'white':matplotlib.colors.to_rgb('#66c2a5'), "asian":matplotlib.colors.to_rgb('#fc8d62'), 'black':matplotlib.colors.to_rgb('#8da0cb')}

def pd_read_pickle_race(path):
    results = pd.read_pickle(path)
    y_label = np.array(results['labels']).astype(float)
    all_races = np.array(results["all_races"])
    
    results_df = pd.DataFrame({'Y': y_label, 
                               'p_0': np.array(results['probs'][:,0]),
                               'p_1': np.array(results['probs'][:,1]),
                               'race': all_races})
    results_df.index = results['slide_ids']
    results_df.index.name = 'slide_id'
    return results_df

def get_tpr_fpr_auc(results_df, mean_fpr):
    fpr, tpr, thresholds = metrics.roc_curve(results_df["Y"], results_df["p_1"], pos_label=1)
    auc = metrics.roc_auc_score(results_df["Y"], results_df["p_1"])

    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0

    return interp_tpr, auc

metric_cols = ['AUC', 'Cutoff', 'Acc',
'Y=0 P', 'Y=0 R', 'Y=0 F1', 'Y=0 Support',
'Y=1 P', 'Y=1 R', 'Y=1 F1', 'Y=1 Support',
'Macro Avg P', 'Macro Avg R', 'Macro Avg F1',
'Weight Avg P', 'Weight Avg R', 'Weight Avg F1'
]

def pd_read_pickle(path):
    results = pd.read_pickle(path)
    y_label = np.array(results['labels']).astype(float)
    
    results_df = pd.DataFrame({'Y': y_label, 
                               'p_0': np.array(results['probs'][:,0]),
                               'p_1': np.array(results['probs'][:,1])})
    results_df.index = results['slide_ids']
    results_df.index.name = 'slide_id'
    return results_df

In [ ]:
from scipy import stats

def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2
    

def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=float)
    T2[J] = T
    return T2

def fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=float)
    ty = np.empty([k, n], dtype=float)
    tz = np.empty([k, m + n], dtype=float)
    for r in range(k):
        tx[r, :] = compute_midrank_weight(positive_examples[r, :], sample_weight[:m])
        ty[r, :] = compute_midrank_weight(negative_examples[r, :], sample_weight[m:])
        tz[r, :] = compute_midrank_weight(predictions_sorted_transposed[r, :], sample_weight)
    total_positive_weights = sample_weight[:m].sum()
    total_negative_weights = sample_weight[m:].sum()
    pair_weights = np.dot(sample_weight[:m, np.newaxis], sample_weight[np.newaxis, m:])
    total_pair_weights = pair_weights.sum()
    aucs = (sample_weight[:m]*(tz[:, :m] - tx)).sum(axis=1) / total_pair_weights
    v01 = (tz[:, :m] - tx[:, :]) / total_negative_weights
    v10 = 1. - (tz[:, m:] - ty[:, :]) / total_positive_weights
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov

def fastDeLong_no_weights(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating
              Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=float)
    ty = np.empty([k, n], dtype=float)
    tz = np.empty([k, m + n], dtype=float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov

def fastDeLong(predictions_sorted_transposed, label_1_count, sample_weight):
    if sample_weight is None:
        return fastDeLong_no_weights(predictions_sorted_transposed, label_1_count)
    else:
        return fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight)

def compute_ground_truth_statistics(ground_truth, sample_weight):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight

def delong_roc_variance(ground_truth, predictions, sample_weight=None):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions = np.asarray(predictions)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov

def get_CI(y_true, y_pred):
    alpha = 0.95
    auc, auc_cov = delong_roc_variance(y_true, y_pred)
    auc_std = np.sqrt(auc_cov)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)
    ci = stats.norm.ppf(lower_upper_q, loc=auc, scale=auc_std)
    ci[ci > 1] = 1
    return ci[0], auc, ci[1], auc_std

In [ ]:
def calc_metrics_binary(y_label, y_prob, cutoff=None, return_pred=False):
    """
    Computing (almost all) binary calculation metrics.
    
    Args:
        - y_label (np.array): (n,)-dim np.array containing ground-truth predictions.
        - y_prob (np.array): (n,)-dim np.array containing probability scores (for y=1).
        - cutoff (int): Whether to use a Yolan's J cutoff (calculated from a model)
        - return_pred (np.array): (n,)-dim np.array containing predictions using Yolan's J.
    Return:
        - results (list): List of binary classification metrics.
    """
    ### AUC
    auc = roc_auc_score(y_label, y_prob)
    
    ### Yolans J
    if cutoff == None:
        fpr, tpr, thresholds = roc_curve(y_label, y_prob)
        J = tpr - fpr
        cutoff = thresholds[np.argmax(J)]
    y_pred = np.array(y_prob > cutoff).astype(int)
    
    ### Classification Report
    out = classification_report(y_label, y_pred, output_dict=True, zero_division=0)
    if return_pred:
        return y_pred, [auc, cutoff, out['accuracy'],
            out['0.0']['precision'], out['0.0']['recall'], out['0.0']['f1-score'], out['0.0']['support'],
            out['1.0']['precision'], out['1.0']['recall'], out['1.0']['f1-score'], out['1.0']['support'],
            out['macro avg']['precision'], out['macro avg']['recall'], out['macro avg']['f1-score'],
            out['weighted avg']['precision'], out['weighted avg']['recall'], out['weighted avg']['f1-score'],
           ]
    else:
        return [auc, cutoff, out['accuracy'],
                out['0.0']['precision'], out['0.0']['recall'], out['0.0']['f1-score'], out['0.0']['support'],
                out['1.0']['precision'], out['1.0']['recall'], out['1.0']['f1-score'], out['1.0']['support'],
                out['macro avg']['precision'], out['macro avg']['recall'], out['macro avg']['f1-score'],
                out['weighted avg']['precision'], out['weighted avg']['recall'], out['weighted avg']['f1-score'],
               ]

def find_interesting_fold(eval_path):
    metrics = []
    y_label_all, y_prob_all = [], []
    for i in range(20):
        
        if os.path.isfile(os.path.join(eval_path, 's_%d_checkpoint_results.pkl' % i)):
            results_df = pd_read_pickle(j_(eval_path, 's_%d_checkpoint_results.pkl' % i))
        else:
            continue

        y_label = np.array(results_df['Y'])
        y_prob = np.array(results_df['p_1'])
        y_label_all.append(y_label)
        y_prob_all.append(y_prob)
        metrics.append(calc_metrics_binary(y_label, y_prob, cutoff=None))


    y_label_all = np.hstack(y_label_all)
    y_prob_all = np.hstack(y_prob_all)
    metrics.append(calc_metrics_binary(y_label_all, y_prob_all, cutoff=None))
    metrics = pd.DataFrame(metrics)
    metrics.columns = metric_cols
    best_fold = metrics['AUC'].argmax()
    worst_fold = metrics['AUC'].argmin()

    return best_fold, worst_fold

In [ ]:
root_results_dir = "/path/to/tcga/validation/results"
models_to_test_all = os.listdir(root_results_dir)
dataroot = 'path/to/test/cohort/results'

In [ ]:
models_to_test = {"ABMIL": [], "CLAM": [], "TMIL":[]}
for model in models_to_test_all:
    if "ABMIL" in model:
        models_to_test["ABMIL"].append(model)
    elif "CLAM" in model:
        models_to_test["CLAM"].append(model)
    else:
        models_to_test["TMIL"].append(model)

In [ ]:
count = 0 
for model in models_to_test["ABMIL"]:
    
    model_name = model.split("_")[2]

    eval_path = j_(dataroot, model)

    # first find the best and worst folds to plot roc's for 
    best_fold, worst_fold = find_interesting_fold(eval_path)
    folds = {"best": best_fold, "worst": worst_fold}
    
    for fold_type in ["best"]:

        tprs_w = []
        aucs_w = []
        mean_fpr_w = np.linspace(0, 1, 500)

        tprs_b = []
        aucs_b = []
        mean_fpr_b = np.linspace(0, 1, 500)

        tprs_a = []
        aucs_a = []
        mean_fpr_a = np.linspace(0, 1, 500)

        fold_to_eval = folds[fold_type]

        # load best fold
        results_df = pd_read_pickle_race(j_(eval_path, 's_%d_checkpoint_results.pkl' % fold_to_eval))

        # white
        tpr_w, auc_w = get_tpr_fpr_auc(results_df[results_df["race"] == 0], mean_fpr_w)
        tprs_w.append(tpr_w)
        aucs_w.append(auc_w)

        # black
        tpr_b, auc_b = get_tpr_fpr_auc(results_df[results_df["race"] == 1], mean_fpr_b)
        tprs_b.append(tpr_b)
        aucs_b.append(auc_b)

        # asian
        tpr_a, auc_a = get_tpr_fpr_auc(results_df[results_df["race"] == 2], mean_fpr_a)
        tprs_a.append(tpr_a)
        aucs_a.append(auc_a)
        
        # get overall mean AUC and CI 
        low_auc, mean_auc, high_auc, std = get_CI(results_df["Y"], results_df["p_1"])
        print("{} = {:.3f} ({:.3f}, {:.3f}) {:.3f}".format(model, mean_auc, low_auc, high_auc, std))
        
        # white 
        mean_tpr_w = np.mean(tprs_w, axis=0)
        mean_tpr_w[-1] = 1.0
        mean_auc_w = np.mean(aucs_w)

        df_sub = results_df[results_df["race"] == 0]
        y_label, y_pred = df_sub["Y"], df_sub["p_1"]

        low_auc_w, mean_auc_w, high_auc_w, std = get_CI(y_label, y_pred)
        print("mean white auc = {:.3f} +/- {:.3f}".format(mean_auc_w, std))

        ci_w = high_auc_w - mean_auc_w
        tprs_upper_w = np.minimum(mean_tpr_w+ci_w, 1)
        tprs_lower_w = np.maximum(mean_tpr_w-ci_w, 0)

        label_w = r'%s (%0.3f $\pm$ %0.3f)' % ("White", mean_auc_w, ci_w)

        # black
        mean_tpr_b = np.mean(tprs_b, axis=0)
        mean_tpr_b[-1] = 1.0
        mean_auc_b = np.mean(aucs_b)

        df_sub = results_df[results_df["race"] == 1]
        y_label, y_pred = df_sub["Y"], df_sub["p_1"]
        low_auc_b, mean_auc_b, high_auc_b, std = get_CI(y_label, y_pred)
        print("mean black auc = {:.3f} +/- {:.3f}".format(mean_auc_b, std))

        ci_b = high_auc_b - mean_auc_b
        tprs_upper_b = np.minimum(mean_tpr_b+ci_b, 1)
        tprs_lower_b = np.maximum(mean_tpr_b-ci_b, 0)

        label_b = r'%s (%0.3f $\pm$ %0.3f)' % ("Black", mean_auc_b, ci_b)

        # asian
        mean_tpr_a = np.mean(tprs_a, axis=0)
        mean_tpr_a[-1] = 1.0
        mean_auc_a = np.mean(aucs_a)

        df_sub = results_df[results_df["race"] == 2]
        y_label, y_pred = df_sub["Y"], df_sub["p_1"]
        low_auc_a, mean_auc_a, high_auc_a, std = get_CI(y_label, y_pred)
        print("mean asian auc = {:.3f} +/- {:.3f}".format(mean_auc_a, std))

        ci_a = high_auc_a - mean_auc_a
        tprs_upper_a = np.minimum(mean_tpr_a+ci_a, 1)
        tprs_lower_a = np.maximum(mean_tpr_a-ci_a, 0)

        label_a = r'%s (%0.3f $\pm$ %0.3f)' % ("Asian", mean_auc_a, ci_a)

        fi = pylab.figure(figsize=(10,10), dpi=600, linewidth=0.2)
        fig, ax = plt.subplots(figsize=(10,10))
        mpl.rcParams['axes.linewidth'] = 3 #set the value globally
        plt.rcParams.update({'font.size':22})

        ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="k", alpha=0.8)

        # white
        plt.plot(mean_fpr_w, mean_tpr_w, color=cdict["white"], label=label_w, lw=4, alpha=0.8)
        plt.fill_between(mean_fpr_w, tprs_lower_w, tprs_upper_w, color=cdict["white"], alpha=0.1)

        # black
        plt.plot(mean_fpr_b, mean_tpr_b, color=cdict["black"], label=label_b, lw=4, alpha=0.8)
        plt.fill_between(mean_fpr_b, tprs_lower_b, tprs_upper_b, color=cdict["black"], alpha=0.1)

        # asian
        plt.plot(mean_fpr_a, mean_tpr_a, color=cdict["asian"], label=label_a, lw=4, alpha=0.8)
        plt.fill_between(mean_fpr_a, tprs_lower_a, tprs_upper_a, color=cdict["asian"], alpha=0.1)

        ax.set_xlabel('1 - Specificity')
        ax.set_ylabel('Sensitivity')
        ax.set_xlim([0.,1.0])
        ax.set_ylim([0.,1.0])
        
        ax.tick_params(axis='both', which='major', labelsize=20)
        ax.set_xticks(np.arange(0, 1.001, 0.2))
        ax.set_yticks(np.arange(0, 1.001, 0.2))
        
        ax.tick_params(        
            which='both',      
            bottom=False,      
            top=False,
            left=False,
            right=False,         
            labelbottom=False,
            labelleft=False) 


        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        ax.axis('off')

        plt.tight_layout()
        ax.legend(loc="lower right")

        print("Done with {} ({})".format(model, fold_type)) 
        count += 1